# Анализ источников новостей

Цель данного исследования: проанализировать источники новостных лент, сравнить рубрики, подумать над возможностью добавление\удаление категорий для проекта "Радар новостей рунета"

In [2]:
import numpy as np
import pandas as pd
import datetime as DT

## tass

Загрузим датасет tass. Одновременно сделаем преобразование даты из строки

In [3]:
dateparse = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [5]:
data_tass = pd.read_csv('../data/tass.csv',
                      sep=',',
                      index_col=False,
                      escapechar='\\',
                      parse_dates=['date'],
                      date_parser=dateparse,
                      quotechar='\"',
                      error_bad_lines=False)

b'Skipping line 75267: expected 17 fields, saw 20\n'
b'Skipping line 232419: expected 17 fields, saw 29\n'
b'Skipping line 409572: expected 17 fields, saw 45\n'
b'Skipping line 485114: expected 17 fields, saw 20\n'
b'Skipping line 551616: expected 17 fields, saw 20\n'
b'Skipping line 567601: expected 17 fields, saw 20\n'
b'Skipping line 607937: expected 17 fields, saw 20\n'
b'Skipping line 710539: expected 17 fields, saw 20\n'
b'Skipping line 947947: expected 17 fields, saw 20\n'
b'Skipping line 995945: expected 17 fields, saw 20\n'
b'Skipping line 1042613: expected 17 fields, saw 20\n'
/home/avenon/anaconda2/envs/proj_news_viz/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
# Проверим преобразование
data_tass.dtypes

date           datetime64[ns]
url                    object
edition                object
topics                 object
authors                object
title                  object
text                   object
reposts_fb             object
reposts_vk             object
reposts_ok             object
reposts_twi            object
reposts_lj             object
reposts_tg             object
likes                  object
views                  object
comm_count             object
dtype: object

In [7]:
data_tass.shape

(1135635, 16)

In [8]:
# Так выглядет датасет tass
data_tass.head()

,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-08-10 17:12:51,https://tass.ru/proisshestviya/6751577,-,proisshestviya,NaN,Военные завершили работы по борьбе с паводками...,"ТАСС, 10 августа. Военнослужащие группировки М...",-,-,-,-,-,-,-,-,-
1,2019-08-10 18:48:42,https://tass.ru/proisshestviya/6751784,-,proisshestviya,NaN,Две ЛЭП повреждены в Ачинске после взрывов на ...,"КРАСНОЯРСК, 10 августа. /ТАСС/. Взрывы на воен...",-,-,-,-,-,-,-,-,-
2,2019-08-10 18:04:35,https://tass.ru/sibir-news/6751662,-,sibir-news,NaN,В двух городах Иркутской области зафиксировано...,"ИРКУТСК, 10 августа. /ТАСС/. Превышение предел...",-,-,-,-,-,-,-,-,-
3,2019-08-08 19:22:59,https://tass.ru/moskovskaya-oblast/6745682,-,moskovskaya-oblast,NaN,Расчистку участка реки Куновки в Талдомском ок...,"МОСКВА, 8 августа. /ТАСС/. Министерство эколог...",-,-,-,-,-,-,-,-,-
4,2019-08-09 13:30:18,https://tass.ru/nauka/6747980,-,nauka,NaN,Ученые смогли определить историю активности зв...,"МОСКВА, 9 августа. /ТАСС/. Международная групп...",-,-,-,-,-,-,-,-,-


In [9]:
# Выясним в каким столбцах есть пустые данные
data_tass.isnull().any()

date           False
url            False
edition        False
topics         False
authors         True
title           True
text            True
reposts_fb     False
reposts_vk     False
reposts_ok     False
reposts_twi    False
reposts_lj     False
reposts_tg     False
likes          False
views          False
comm_count     False
dtype: bool

Как видим, в некоторых столбцах вместо значения Nan используется "-", их преобразованием займемся позже. А пока посмотрим строки, в которых пустые тексты новостей

In [10]:
print('Кол-во строк с значением NaN в столбце "authors" = ', data_tass['authors'].isnull().sum())
data_tass[data_tass['authors'].isnull()]

Кол-во строк с значением NaN в столбце "authors" =  1134298


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-08-10 17:12:51,https://tass.ru/proisshestviya/6751577,-,proisshestviya,NaN,Военные завершили работы по борьбе с паводками...,"ТАСС, 10 августа. Военнослужащие группировки М...",-,-,-,-,-,-,-,-,-
1,2019-08-10 18:48:42,https://tass.ru/proisshestviya/6751784,-,proisshestviya,NaN,Две ЛЭП повреждены в Ачинске после взрывов на ...,"КРАСНОЯРСК, 10 августа. /ТАСС/. Взрывы на воен...",-,-,-,-,-,-,-,-,-
2,2019-08-10 18:04:35,https://tass.ru/sibir-news/6751662,-,sibir-news,NaN,В двух городах Иркутской области зафиксировано...,"ИРКУТСК, 10 августа. /ТАСС/. Превышение предел...",-,-,-,-,-,-,-,-,-
3,2019-08-08 19:22:59,https://tass.ru/moskovskaya-oblast/6745682,-,moskovskaya-oblast,NaN,Расчистку участка реки Куновки в Талдомском ок...,"МОСКВА, 8 августа. /ТАСС/. Министерство эколог...",-,-,-,-,-,-,-,-,-
4,2019-08-09 13:30:18,https://tass.ru/nauka/6747980,-,nauka,NaN,Ученые смогли определить историю активности зв...,"МОСКВА, 9 августа. /ТАСС/. Международная групп...",-,-,-,-,-,-,-,-,-
5,2019-08-09 15:37:53,https://tass.ru/nauka/6748360,-,nauka,NaN,В МГУ узнали причину неостанавливаемых кровоте...,"МОСКВА, 9 августа. /ТАСС/. Ученые МГУ им. Ломо...",-,-,-,-,-,-,-,-,-
6,2019-08-09 05:56:49,https://tass.ru/obschestvo/6747431,-,obschestvo,NaN,Джонсон поручил разработать систему упрощенног...,"ЛОНДОН, 9 августа. /ТАСС/. Премьер-министр Вел...",-,-,-,-,-,-,-,-,-
7,2019-08-11 19:54:01,https://tass.ru/proisshestviya/6753361,-,proisshestviya,NaN,На Канарах 1 тыс. человек эвакуировали из-за с...,"ТАСС, 11 августа. Число эвакуированных из-за в...",-,-,-,-,-,-,-,-,-
8,2019-08-11 20:09:43,https://tass.ru/proisshestviya/6753382,-,proisshestviya,NaN,Reuters: число жертв ливней и наводнений на юг...,"ТАСС, 11 августа. Не менее 147 человек погибли...",-,-,-,-,-,-,-,-,-
9,2019-08-09 07:34:01,https://tass.ru/armiya-i-opk/6747493,-,armiya-i-opk,NaN,На Камчатку вернулся после четырехлетнего ремо...,"ПЕТРОПАВЛОВСК-КАМЧАТСКИЙ, 9 августа. /ТАСС/. А...",-,-,-,-,-,-,-,-,-


In [12]:
print('Кол-во строк с значением NaN в столбце "text" = ', data_tass['text'].isnull().sum())
data_tass[data_tass['text'].isnull()]

Кол-во строк с значением NaN в столбце "text" =  19


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
13062,2018-11-21 14:41:16,https://tass.ru/ekonomika/5812803,-,ekonomika,NaN,Я хочу открыть бизнес. Как мне может помочь По...,NaN,-,-,-,-,-,-,-,-,-
27794,2019-07-03 21:18:25,https://tass.ru/proisshestviya/6615831,-,proisshestviya,NaN,Двух врачей обвиняют в убийстве младенца.,NaN,-,-,-,-,-,-,-,-,-
75906,2019-04-25 15:15:01,https://tass.ru/kultura/6367489,-,kultura,NaN,Как долго вы продержитесь на необитаемом острове?,NaN,-,-,-,-,-,-,-,-,-
349884,2017-02-01 21:49:26,https://tass.ru/v-strane/3988667,-,v-strane,NaN,"""Дальневосточный гектар"": что нужно знать о бе...",NaN,-,-,-,-,-,-,-,-,-
353315,2016-11-01 01:00:00,https://tass.ru/infographics/8320,-,infographics,NaN,Развитие массового спорта на Северном Кавказе,NaN,-,-,-,-,-,-,-,-,-
387409,2014-05-14 11:21:15,https://tass.ru/sibir-news/1184635,-,sibir-news,NaN,В Красноярском крае площадь природных пожаров ...,NaN,-,-,-,-,-,-,-,-,-
389177,2014-04-16 14:18:50,https://tass.ru/sibir-news/1124718,-,sibir-news,NaN,"В Красноярске создают спутник, который отправи...",NaN,-,-,-,-,-,-,-,-,-
390152,2014-04-06 08:45:20,https://tass.ru/sibir-news/1100309,-,sibir-news,NaN,В Омской области в жилом доме произошел взрыв ...,NaN,-,-,-,-,-,-,-,-,-
390154,2014-04-06 02:38:52,https://tass.ru/sibir-news/1100170,-,sibir-news,NaN,В Томской области паводок подтопил жилые дома,NaN,-,-,-,-,-,-,-,-,-
446478,2015-02-02 17:43:41,https://tass.ru/politika/1738710,-,politika,NaN,Верховный суд РФ 4 марта рассмотрит жалобу на ...,NaN,-,-,-,-,-,-,-,-,-


In [13]:
print('Кол-во строк с значением NaN в столбце "title" = ', data_tass['title'].isnull().sum())
data_tass[data_tass['title'].isnull()]

Кол-во строк с значением NaN в столбце "title" =  1


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
680568,2016-08-10 18:36:16,https://tass.ru/proisshestviya/3525598,-,proisshestviya,NaN,NaN,"КРАСНОДАР, 10 августа. /Корр. ТАСС Сергей Окол...",-,-,-,-,-,-,-,-,-


In [14]:
print('authors: ',data_tass['authors'].unique())

authors:  [nan 'ХАНОВ Михаил\xa0' 'ЛИТОВКИН Виктор\xa0' 'МЕЗЕНЦЕВА Елена\xa0'
 'МАМОНТОВ Андрей\xa0' 'ЧЕКАЛОВА Елена\xa0' 'МОРЖАРЕТТО Игорь\xa0'
 'НАЗАЙКИНСКАЯ Ольга\xa0' 'МАНСКАЯ Полина\xa0' 'ГРОМЫКО Алексей\xa0'
 'КОЛОБКОВ Павел\xa0' 'ЖУКОВ Владислав\xa0' 'СТАРОДУБЦЕВ Альберт\xa0'
 'КИРИЛЛОВ Андрей\xa0' 'КУЗНЕЦОВ Артем\xa0' 'ШИРОВ Александр\xa0'
 'КУРСАКОВА Мария\xa0' 'ПОРЯДИНА  Мария\xa0' 'ВАСИЛЬЕВ Александр\xa0'
 'ХОДОВА Тамара\xa0' 'МОРСИН Александр\xa0' 'ДОЛГИН Николай\xa0'
 'ЛАНОВОЙ Василий\xa0' 'ШИТОВ Андрей\xa0' 'ГОРОХОВ Дмитрий\xa0'
 'ШАЛАШОВА Анна\xa0' 'ЛИ Хуэй\xa0' 'ТЮРИНА Анна\xa0' 'БИРГЕР Лиза\xa0'
 'ОКУДЖАВА Ольга\xa0' 'ХЛОПОВА Вита\xa0' 'ЖИТИНКИН Андрей \xa0'
 'ЖУРАВЛЕВА Евгения\xa0' 'СТЕБЛОВ Евгений\xa0' 'МАМЕД-ЗАДЕ Первин\xa0'
 'МАКАРОВА Ая\xa0' 'СИНИЦЫН Сергей\xa0' 'ЩЕРБАКОВА Вера\xa0'
 'ЖИГУНОВ Сергей\xa0' 'СМЕХОВ Вениамин\xa0' 'ЗОЛОТОВИЦКИЙ Игорь \xa0'
 'ТУМИНАС Римас \xa0' 'ДЖОПУА Руслан\xa0' 'ГАЛСТЯН Арег\xa0'
 'СОЛОВЬЕВ Евгений\xa0' 'ЗАХАРОВ Марк\xa0' 'МАМЕДОВ Р

Посмотрим, что в столбцах, которые содержат дефис, как обозначение пустых данных

In [15]:
print('edition: ',data_tass['edition'].unique())
print('reposts_fb: ', data_tass['reposts_fb'].unique())
print('reposts_vk: ',data_tass['reposts_vk'].unique())
print('reposts_ok: ',data_tass['reposts_ok'].unique())
print('reposts_twi: ',data_tass['reposts_twi'].unique())
print('reposts_lj: ',data_tass['reposts_lj'].unique())
print('reposts_tg: ',data_tass['reposts_tg'].unique())
print('likes: ',data_tass['likes'].unique())
print('views: ',data_tass['views'].unique())
print('comm_count: ',data_tass['comm_count'].unique())

edition:  ['-']
reposts_fb:  ['-']
reposts_vk:  ['-']
reposts_ok:  ['-']
reposts_twi:  ['-']
reposts_lj:  ['-']
reposts_tg:  ['-']
likes:  ['-']
views:  ['-']
comm_count:  ['-']


### Рубрики tass

In [16]:
# Считаем количество рубрик в данном датасете, а также выведем значения
print('Количество рубрик = ', data_tass.topics.unique().shape)
data_tass['topics'].unique()

Количество рубрик =  (288,)


array(['proisshestviya', 'sibir-news', 'moskovskaya-oblast', 'nauka',
       'obschestvo', 'armiya-i-opk', 'nedvizhimost', 'moskva',
       'nacionalnye-proekty', 'ekonomika', 'mezhdunarodnaya-panorama',
       'v-strane', 'kultura', 'tehnologii', 'politika', 'ural-news',
       'zaderzhanie-eks-prezidenta-kirgizii-atambaeva', 'obrazovanie',
       'opinions', 'interviews', 'msp', 'renovaciya', 'infographics',
       'kosmos', 'old-ekonomika', 'spb-news', 'sport', 'severnyy-kavkaz',
       'religiya', 'arktika-segodnya', 'info',
       'delo-v-otnoshenii-eks-ministra-mihaila-abyzova',
       'oprosy-obschestvennogo-mneniya', 'svoy-buziness', 'msp-analitika',
       'msp-foto', 'bank-istoriy-uspeha', 'msp-video',
       'novosti-partnerov', 'nacproekt-ekologiya', 'msp-intervyu',
       'online-conference', 'transport', 'vopros-otvet',
       'msp-kommentarii', 'msp-reportazhi', 'onlayn-konferencii', 'tek',
       'maloe-i-srednee-predprinimatelstvo', 'cifrovaya-energetika',
       'opyt

Все рубрики, с количеством новостей в каждой группе

In [17]:
data_tass.groupby('topics')['topics'].count()

topics
110-let-tass                           13
70-letie-pobedy                         5
akcii-protesta-v-tailande               1
akcii-protesta-vo-francii               1
akciya-greenpeace-na-prirazlomnoy      20
                                       ..
zakrytie-svalki-v-balashihe             2
zdorove                                 8
zdravoohranenie                         3
zemletryasenie-i-cunami-v-indonezii     2
zimnie-vidy                             5
Name: topics, Length: 288, dtype: int64